LDA (LatentDirichletAllocation): [link](https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0)

In [1]:
import pandas as pd
import re

from sklearn.datasets import fetch_20newsgroups

In [2]:
seed = 94487

In [12]:
df = pd.read_csv('./data/water_problem_nlp_en_for_Kaggle_100.csv',sep=';')
df.sample(5, random_state=seed)

,text,env_problems,pollution,treatment,climate,biomonitoring
47,Main part (86%) of phosphate ions in wastewate...,1,1.0,NaN,NaN,NaN
36,The order of values of its organic forms estab...,1,1.0,NaN,NaN,1.0
62,Southern Bug basin is located in the right-ban...,0,NaN,NaN,NaN,NaN
90,Ladyzhyn water storage reservoir due to its bi...,1,1.0,NaN,NaN,1.0
65,At Podilska and Dnieper Elevations relief is f...,0,NaN,NaN,NaN,NaN


In [28]:
# train = fetch_20newsgroups(subset='train')
# test = fetch_20newsgroups(subset='test')

In [13]:
# df = pd.DataFrame({'text':train.data})
# df.sample(5, random_state=seed)

In [14]:
# Remove punctuation
df['text_processed'] = \
df['text'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
df['text_processed'] = \
df['text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
df.sample(5, random_state=seed)

,text,env_problems,pollution,treatment,climate,biomonitoring,text_processed
47,Main part (86%) of phosphate ions in wastewate...,1,1.0,NaN,NaN,NaN,main part (86%) of phosphate ions in wastewate...
36,The order of values of its organic forms estab...,1,1.0,NaN,NaN,1.0,the order of values of its organic forms estab...
62,Southern Bug basin is located in the right-ban...,0,NaN,NaN,NaN,NaN,southern bug basin is located in the right-ban...
90,Ladyzhyn water storage reservoir due to its bi...,1,1.0,NaN,NaN,1.0,ladyzhyn water storage reservoir due to its bi...
65,At Podilska and Dnieper Elevations relief is f...,0,NaN,NaN,NaN,NaN,at podilska and dnieper elevations relief is f...


In [15]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.text_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['southern', 'bug', 'basin', 'wastewater', 'discharged', 'organized', 'sources', 'map', 'reference', 'year', 'mln', 'discharged', 'total']


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/carlosmorote/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)]


In [23]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 5 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.019*"oc" + 0.016*"observed" + 0.015*"temperature" + 0.012*"river" + '
  '0.012*"vinnitsya" + 0.012*"khmelnitsky" + 0.012*"wwtps" + '
  '0.012*"downstream" + 0.012*"volumes" + 0.012*"wastewater"'),
 (1,
  '0.024*"basin" + 0.021*"water" + 0.021*"bodies" + 0.014*"southern" + '
  '0.014*"bug" + 0.014*"organic" + 0.014*"phosphorus" + 0.014*"within" + '
  '0.014*"discharged" + 0.014*"kg"'),
 (2,
  '0.022*"nitrogen" + 0.021*"part" + 0.015*"waters" + 0.012*"also" + '
  '0.012*"lower" + 0.012*"basin" + 0.011*"surface" + 0.011*"average" + '
  '0.009*"vinnistya" + 0.009*"inorganic"'),
 (3,
  '0.042*"nitrogen" + 0.024*"part" + 0.019*"basin" + 0.014*"main" + '
  '0.014*"tons" + 0.012*"forms" + 0.010*"compounds" + 0.010*"wastewater" + '
  '0.010*"khmelnitsky" + 0.010*"kirovograd"'),
 (4,
  '0.020*"wastewater" + 0.014*"days" + 0.014*"values" + 0.014*"temperature" + '
  '0.014*"settlements" + 0.011*"water" + 0.010*"tons" + 0.010*"oc" + '
  '0.010*"many" + 0.010*"treatment"'),
 (5,
  '0.045*"

In [24]:
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

/Users/carlosmorote/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.003869  0.020959       1        1  13.746862
2     -0.122710 -0.031680       2        1  12.258615
4      0.000382  0.112989       3        1  11.237471
3     -0.124829 -0.060910       4        1  10.983154
6     -0.011373 -0.074110       5        1  10.932618
7     -0.029452  0.093156       6        1  10.462514
5      0.128762 -0.089442       7        1  10.291493
0      0.056467  0.080218       8        1   8.628271
1      0.063052 -0.017202       9        1   5.875480
8      0.043569 -0.033978      10        1   5.583522, topic_info=           Term       Freq      Total Category  logprob  loglift
40     nitrogen  14.000000  14.000000  Default  30.0000  30.0000
0         basin  19.000000  19.000000  Default  29.0000  29.0000
56        water  15.000000  15.000000  Default  28.0000  28.0000
476     climate   4.000000   4.000000  Default  27.0000  27.0000
1           bug  13.000000  13.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
167  phosphorus   0.690316   4.438039  Topic10  -4.7223   1.0245
41      organic   0.669735   4.520430  Topic10  -4.7526   0.9759
187      values   0.670346   5.603550  Topic10  -4.7517   0.7620
30         part   0.690302  15.854701  Topic10  -4.7223  -0.2487
8      southern   0.649693  12.712788  Topic10  -4.7830  -0.0885

[566 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
94        2  0.713412    accidents
226       9  0.821840  accordingly
123       6  0.752702     accurate
525       3  0.712745       active
183       8  0.728499   additional
...     ...       ...          ...
11        2  0.162570         year
11        3  0.162570         year
11        4  0.162570         year
11        5  0.162570         year
11        9  0.162570         year

[756 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 3, 5, 4, 7, 8, 6, 1, 2, 9])

In [34]:
from gensim.test.utils import common_corpus, common_dictionary

# TODO: No usar el common_dictionary. Hay que extrapolar o generalizar exactamente el mismo que para los modelos. Tal vez con eso lo que devuelva tenga algo de sentido también.

interest_themes = [['human']]
interest_themes_corpus = [common_dictionary.doc2bow(text) for text in interest_themes]
sorted(lda_model[interest_themes_corpus[0]], key=lambda x: x[1], reverse=True)

[(8, 0.5499279),
 (6, 0.050020903),
 (5, 0.05001715),
 (1, 0.050012782),
 (3, 0.05000857),
 (7, 0.05000403),
 (2, 0.050003275),
 (4, 0.05000265),
 (9, 0.050002478),
 (0, 0.050000336)]